In [1]:
#import relevant libraries
import io
import itertools

import sklearn.metrics
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [3]:
#loading the dataset
train_data = np.load(r'datasets/Primary categories - Train.npz')
test_data = np.load(r'datasets/Primary categories - Test.npz')
val_data = np.load(r'datasets/Primary categories - Validation.npz')

In [9]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [10]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [11]:
#constant definition
NUM_EPOCHS = 15
BATCH_SIZE = 64

In [12]:
#hyperparam definition
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_NUM_FILTER = hp.HParam('filters_number', hp.Discrete([32,64,96,128]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_FILTER_SIZE,HP_NUM_FILTER],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [23]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTER],3,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'Logs\\Model 1\\fit\\'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses', 'Trousers/Jeans', 'Shoes'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'saved Models\Model 1\Run-{}'.format(session_num))
    
    return accuracy
    
    
    

In [24]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [25]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for num_filter in HP_NUM_FILTER.domain.values:
        
        hparams = {
            HP_FILTER_SIZE : filter_size,
            HP_NUM_FILTER : num_filter
        }
        
        run_name = 'run-%d' %session_num
        print('---- Starting trial : %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' +run_name, hparams, session_num)
        
        session_num += 1

---- Starting trial : run-1
{'filter_size': 3, 'filters_number': 32}
Epoch 1/15
203/203 - 56s - loss: 0.0814 - accuracy: 0.9737 - val_loss: 0.0062 - val_accuracy: 0.9988
Epoch 2/15
203/203 - 58s - loss: 0.0115 - accuracy: 0.9987 - val_loss: 0.0060 - val_accuracy: 0.9994
Epoch 3/15
203/203 - 55s - loss: 0.0070 - accuracy: 0.9995 - val_loss: 0.0020 - val_accuracy: 0.9994
Epoch 4/15
203/203 - 55s - loss: 0.0064 - accuracy: 0.9988 - val_loss: 0.0025 - val_accuracy: 0.9994
Epoch 5/15
203/203 - 58s - loss: 0.0055 - accuracy: 0.9992 - val_loss: 0.0266 - val_accuracy: 0.9938
51/51 [==============================] - 2s 36ms/step - loss: 0.0020 - accuracy: 0.9994
INFO:tensorflow:Assets written to: saved Models\Model 1\Run-1\assets
---- Starting trial : run-2
{'filter_size': 3, 'filters_number': 64}
Epoch 1/15
203/203 - 119s - loss: 0.0858 - accuracy: 0.9755 - val_loss: 0.0092 - val_accuracy: 0.9994
Epoch 2/15
203/203 - 117s - loss: 0.0152 - accuracy: 0.9982 - val_loss: 0.0126 - val_accuracy: 0.9

In [28]:
model = tf.keras.models.load_model(r"saved models\Model 1\Run-1")

In [29]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

51/51 [==============================] - 2s 33ms/step - loss: 0.0208 - accuracy: 0.9975


In [30]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.0208. Test accuracy: 99.75%


In [31]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hparam_tuning"

In [33]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
